In [ ]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler


# VIOLATIONS DATA

In [ ]:
viol_data = pd.read_csv('/home/deploy/notebooks/city_violations_data/input/SF_download/violations/Notices_of_violation_SF.csv',
                sep = ',')

In [ ]:
viol_data.shape

In [ ]:
# wrangling some columns

# converting column to date
viol_data['date'] = pd.to_datetime(viol_data['Date Filed'], format='%m/%d/%Y')

# creating column which will be a basis for prediction - block_lot (parcel)
viol_data['block_lot'] = viol_data['Block'].astype(str) + '_' + viol_data['Lot'].astype(str)

In [ ]:
viol_data.drop(['Date Filed','Complaint Number', 'Item Sequence Number', 'Street Suffix', 'Unit','Receiving Division','Assigned Division',
        'Zipcode', 'NOV Item Description',  ], axis =1 , inplace = True   )

In [ ]:
viol_data.head(5)

In [ ]:
for col in viol_data.columns:
    print('{}: {}'.format(col, len(viol_data[col].unique())))

In [ ]:
for col in viol_data.columns:
    print(col, viol_data[col].isna().sum())

### group level feature engineering

In [ ]:
viol_df = viol_data.copy()
viol_df = viol_df.sort_values(by='date')

#################################### TIER I DERIVATED VARIABLES ####################################

# A) calculating days since last violations for 3 most recent violations ocurred 
auxiliary_viol_df = viol_df[['block_lot', 'date']].drop_duplicates()
auxiliary_viol_df['date_shift_one'] = auxiliary_viol_df.groupby(['block_lot'])['date'].shift(+1)
auxiliary_viol_df['date_shift_two'] = auxiliary_viol_df.groupby(['block_lot'])['date'].shift(+2)
auxiliary_viol_df['date_shift_three'] = auxiliary_viol_df.groupby(['block_lot'])['date'].shift(+3)
auxiliary_viol_df['date_shift_four'] = auxiliary_viol_df.groupby(['block_lot'])['date'].shift(+4)
auxiliary_viol_df['date_shift_five'] = auxiliary_viol_df.groupby(['block_lot'])['date'].shift(+5)

viol_df = viol_df.merge(auxiliary_viol_df, on = ['block_lot', 'date'], how = 'left')

# calculating timedelta
viol_df['days_since_last_viol'] = viol_df['date'] - viol_df['date_shift_one']
viol_df['days_since_last_viol_shift'] = viol_df['date'] - viol_df['date_shift_two']
viol_df['days_since_last_viol_shift_two'] = viol_df['date'] - viol_df['date_shift_three']
viol_df['days_since_last_viol_shift_three'] = viol_df['date'] - viol_df['date_shift_four']
viol_df['days_since_last_viol_shift_four'] = viol_df['date'] - viol_df['date_shift_five']

# converting to integer
viol_df['days_since_last_viol'] = viol_df['days_since_last_viol'].dt.days
viol_df['days_since_last_viol_shift'] =viol_df['days_since_last_viol_shift'].dt.days
viol_df['days_since_last_viol_shift_two'] = viol_df['days_since_last_viol_shift_two'].dt.days
viol_df['days_since_last_viol_shift_three'] = viol_df['days_since_last_viol_shift_three'].dt.days
viol_df['days_since_last_viol_shift_four'] = viol_df['days_since_last_viol_shift_four'].dt.days

# replacing null values with zero
viol_df['days_since_last_viol'] = viol_df['days_since_last_viol'].replace(np.nan, 0)
viol_df['days_since_last_viol_shift'] = viol_df['days_since_last_viol_shift'].replace(np.nan, 0)
viol_df['days_since_last_viol_shift_two'] = viol_df['days_since_last_viol_shift_two'].replace(np.nan, 0)
viol_df['days_since_last_viol_shift_three'] = viol_df['days_since_last_viol_shift_three'].replace(np.nan, 0)
viol_df['days_since_last_viol_shift_four'] = viol_df['days_since_last_viol_shift_four'].replace(np.nan, 0)


# B) creating number of viol for 3 most recent violations ocurred  
auxiliary_viol_df=  viol_df.groupby(['block_lot', 'date']).size().reset_index(name='N_violations_current_insp')
auxiliary_viol_df['N_violations_last_insp'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_current_insp'].shift(+1)
auxiliary_viol_df['N_violations_last_insp_shift'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_current_insp'].shift(+2)
auxiliary_viol_df['N_violations_last_insp_shift_two'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_current_insp'].shift(+3)
auxiliary_viol_df['N_violations_last_insp_shift_three'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_current_insp'].shift(+4)
auxiliary_viol_df['N_violations_last_insp_shift_four'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_current_insp'].shift(+5)

viol_df = viol_df.merge(auxiliary_viol_df, on = ['block_lot', 'date'], how = 'left')

# replacing null values with zero
viol_df['N_violations_current_insp'] = viol_df['N_violations_current_insp'].replace(np.nan, 0)
viol_df['N_violations_last_insp'] = viol_df['N_violations_last_insp'].replace(np.nan, 0)
viol_df['N_violations_last_insp_shift'] = viol_df['N_violations_last_insp_shift'].replace(np.nan, 0)
viol_df['N_violations_last_insp_shift_two'] = viol_df['N_violations_last_insp_shift_two'].replace(np.nan, 0)
viol_df['N_violations_last_insp_shift_three'] = viol_df['N_violations_last_insp_shift_three'].replace(np.nan, 0)
viol_df['N_violations_last_insp_shift_four'] = viol_df['N_violations_last_insp_shift_four'].replace(np.nan, 0)


# C) creating cummulative violations till date for 3 most recent violations ocurred 
auxiliary_viol_df['violations_cumsum_till_date'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_current_insp'].cumsum()
auxiliary_viol_df['violations_cumsum_till_date_shift'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_last_insp'].cumsum()
auxiliary_viol_df['violations_cumsum_till_date_shift_two'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_last_insp_shift'].cumsum()
auxiliary_viol_df['violations_cumsum_till_date_shift_three'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_last_insp_shift_two'].cumsum()
auxiliary_viol_df['violations_cumsum_till_date_shift_four'] = auxiliary_viol_df.groupby(['block_lot'])['N_violations_last_insp_shift_three'].cumsum()

# dropping duplicate columns
auxiliary_viol_df = auxiliary_viol_df.drop([
    'N_violations_current_insp',
    'N_violations_last_insp',
    'N_violations_last_insp_shift',
    'N_violations_last_insp_shift_two',
    'N_violations_last_insp_shift_three',
    'N_violations_last_insp_shift_four'],
    axis =1)
viol_df = viol_df.merge(auxiliary_viol_df, on = ['block_lot', 'date'], how = 'left')

# replacing null values with zero
viol_df['violations_cumsum_till_date'] = viol_df['violations_cumsum_till_date'].replace(np.nan, 0)
viol_df['violations_cumsum_till_date_shift'] = viol_df['violations_cumsum_till_date_shift'].replace(np.nan, 0)
viol_df['violations_cumsum_till_date_shift_two'] = viol_df['violations_cumsum_till_date_shift_two'].replace(np.nan, 0)
viol_df['violations_cumsum_till_date_shift_three'] = viol_df['violations_cumsum_till_date_shift_three'].replace(np.nan, 0)
viol_df['violations_cumsum_till_date_shift_four'] = viol_df['violations_cumsum_till_date_shift_four'].replace(np.nan, 0)



#################################### TIER 2 DERIVATED VARIABLES ####################################
# helper lists
days_since_cols = ['days_since_last_viol','days_since_last_viol_shift','days_since_last_viol_shift_two',
                      'days_since_last_viol_shift_three','days_since_last_viol_shift_four']

N_violations_recent_insp_cols = ['N_violations_current_insp','N_violations_last_insp','N_violations_last_insp_shift',
                                 'N_violations_last_insp_shift_two','N_violations_last_insp_shift_three',
                                 'N_violations_last_insp_shift_four']

# creating overal averages and stdevs
viol_df['avg_days_between_previous_violations'] = viol_df[days_since_cols].mean(axis = 1)
viol_df['avg_N_violations_recent_inspections'] = viol_df[N_violations_recent_insp_cols].mean(axis = 1)
viol_df['StDev_days_between_previous_violations'] = viol_df[days_since_cols].std(axis = 1)
viol_df['StDev_days_between_previous_violations'] = viol_df[days_since_cols].std(axis = 1)
viol_df['ratio_avg_days_avg_N_violations'] = viol_df['avg_days_between_previous_violations']/viol_df['avg_N_violations_recent_inspections']

# creating individual vs rest ratios
for col in days_since_cols:
    viol_df["{}_ratio_other".format(col)] = viol_df[col] / viol_df[[x for x in days_since_cols if x!= col]].mean(axis=1)
    viol_df["{}_ratio_other".format(col)]= viol_df["{}_ratio_other".format(col)].replace(np.nan, 0)

for col in N_violations_recent_insp_cols:
    viol_df["{}_ratio_other".format(col)] = viol_df[col] / viol_df[[x for x in N_violations_recent_insp_cols if x!= col]].mean(axis=1)
    viol_df["{}_ratio_other".format(col)] = viol_df["{}_ratio_other".format(col)] .replace(np.nan, 0)

# adding columnwise subtractions
auxiliary_viol_df = viol_df[days_since_cols]
auxiliary_viol_df = auxiliary_viol_df.shift(-1, axis = 1) - auxiliary_viol_df 
auxiliary_viol_df = auxiliary_viol_df.iloc[:, :-1]
auxiliary_viol_df.columns = ["{}_subtracted".format(x) for x in auxiliary_viol_df.columns]
auxiliary_viol_df['AVG_days_since_columnwise_diff'] = auxiliary_viol_df.mean(axis = 1)
auxiliary_viol_df['StDev_days_since_columnwise_diff'] = auxiliary_viol_df.std(axis = 1)
viol_df = pd.concat([viol_df, auxiliary_viol_df], axis =1)

auxiliary_viol_df = viol_df[N_violations_recent_insp_cols]
auxiliary_viol_df = auxiliary_viol_df.iloc[:, :-1]
auxiliary_viol_df = auxiliary_viol_df.shift(-1, axis = 1) - auxiliary_viol_df 
auxiliary_viol_df.columns = ["{}_subtracted".format(x) for x in auxiliary_viol_df.columns]
auxiliary_viol_df['AVG_N_violations_columnwise_diff'] = auxiliary_viol_df.mean(axis = 1)
auxiliary_viol_df['StDev_N_violations_columnwise_diff'] = auxiliary_viol_df.std(axis = 1)
viol_df = pd.concat([viol_df, auxiliary_viol_df], axis =1)

auxiliary_viol_df['AVG_days_since_N_violations_columnwise_diff'] =viol_df['AVG_days_since_columnwise_diff']\
                                                            /viol_df['AVG_N_violations_columnwise_diff']

# adding columnwise ratios
auxiliary_viol_df = viol_df[days_since_cols]
auxiliary_viol_df = auxiliary_viol_df.shift(-1, axis = 1)/auxiliary_viol_df 
auxiliary_viol_df = auxiliary_viol_df.iloc[:, :-1]
auxiliary_viol_df.columns = ["{}_ratio_previous".format(x) for x in auxiliary_viol_df.columns]
viol_df["{}_ratio_other".format(col)] = viol_df["{}_ratio_other".format(col)] .replace(np.nan, 0)

auxiliary_viol_df['AVG_days_since_columnwise_ratio'] = auxiliary_viol_df.mean(axis = 1)
auxiliary_viol_df['StDev_days_since_columnwise_ratio'] = auxiliary_viol_df.std(axis = 1)
viol_df = pd.concat([viol_df, auxiliary_viol_df], axis =1)

auxiliary_viol_df = viol_df[N_violations_recent_insp_cols]
auxiliary_viol_df = auxiliary_viol_df.shift(-1, axis = 1)/auxiliary_viol_df 
auxiliary_viol_df = auxiliary_viol_df.iloc[:, :-1]
auxiliary_viol_df.columns = ["{}_ratio_previous".format(x) for x in auxiliary_viol_df.columns]
auxiliary_viol_df['AVG_N_violations_columnwise_ratio'] = auxiliary_viol_df.mean(axis = 1)
auxiliary_viol_df['StDev_N_violations_columnwise_ratio'] = auxiliary_viol_df.std(axis = 1)
viol_df = pd.concat([viol_df, auxiliary_viol_df], axis =1)

cum_sum_cols = ['violations_cumsum_till_date', 'violations_cumsum_till_date_shift','violations_cumsum_till_date_shift_two',
                'violations_cumsum_till_date_shift_three','violations_cumsum_till_date_shift_four']
auxiliary_viol_df = viol_df[cum_sum_cols]
auxiliary_viol_df = auxiliary_viol_df/auxiliary_viol_df.shift(-1, axis = 1) 
auxiliary_viol_df = auxiliary_viol_df.iloc[:, :-1]
auxiliary_viol_df.columns = ["{}_ratio_previous".format(x) for x in auxiliary_viol_df.columns]
auxiliary_viol_df['AVG_cumsum_columnwise_ratio'] = auxiliary_viol_df.mean(axis = 1)
auxiliary_viol_df['StDev_cumsum_columnwise_ratio'] = auxiliary_viol_df.std(axis = 1)
viol_df = pd.concat([viol_df, auxiliary_viol_df], axis =1)

# adding inter group columnwise ratios
viol_df['ratio_days_since_N_violations_last_insp'] = viol_df['days_since_last_viol']/viol_df['N_violations_last_insp']
viol_df['ratio_days_since_N_violations_last_insp_shift'] = viol_df['days_since_last_viol_shift']/viol_df['N_violations_last_insp_shift']
viol_df['ratio_days_since_N_violations_last_insp_shift_two'] = viol_df['days_since_last_viol_shift_two']/viol_df['N_violations_last_insp_shift_two']
viol_df['ratio_days_since_N_violations_last_insp_shift_three'] = viol_df['days_since_last_viol_shift_three']/viol_df['N_violations_last_insp_shift_three']
viol_df['ratio_days_since_N_violations_last_insp_shift_four'] = viol_df['days_since_last_viol_shift_four']/viol_df['N_violations_last_insp_shift_four']


viol_df['AVG_columnwise_ratio_days_since_N_violations'] = viol_df[['ratio_days_since_N_violations_last_insp',
                                                          'ratio_days_since_N_violations_last_insp_shift_two',
                                                          'ratio_days_since_N_violations_last_insp_shift_three',
                                                          'ratio_days_since_N_violations_last_insp_shift_four']].mean(axis = 1)

viol_df['StDev_columnwise_ratio_days_since_N_violations'] = viol_df[['ratio_days_since_N_violations_last_insp',
                                                          'ratio_days_since_N_violations_last_insp_shift_two',
                                                          'ratio_days_since_N_violations_last_insp_shift_three',
                                                          'ratio_days_since_N_violations_last_insp_shift_four']].std(axis = 1)



### individual category level feature engineering

In [ ]:
#################################### TIER I DERIVATED VARIABLES ####################################

# A) calculating days since last violations for 3 most recent violations ocurred 
for i in viol_df['NOV Category Description'].unique():
    print(i)
    
    auxiliary_viol_df = viol_df[['block_lot', 'date','NOV Category Description']]
    auxiliary_viol_df['{}_date'.format(i)]= np.where(auxiliary_viol_df['NOV Category Description'] == i, 1, np.nan)
    
    auxiliary_viol_df = auxiliary_viol_df.dropna(subset= ['{}_date'.format(i)])
    auxiliary_viol_df['{}_date'.format(i)] = auxiliary_viol_df['date']
    auxiliary_viol_df = auxiliary_viol_df.drop_duplicates(subset = ['block_lot', 'date'])
    auxiliary_viol_df = auxiliary_viol_df.drop(['NOV Category Description'], axis =1)

    auxiliary_viol_df['{}_date_shift'.format(i)] = auxiliary_viol_df.groupby(['block_lot'])['{}_date'.format(i)].shift(+1)
    auxiliary_viol_df['{}_date_shift_two'.format(i)] = auxiliary_viol_df.groupby(['block_lot'])['{}_date'.format(i)].shift(+2)
    auxiliary_viol_df['{}_date_shift_three'.format(i)] = auxiliary_viol_df.groupby(['block_lot'])['{}_date'.format(i)].shift(+3)
    auxiliary_viol_df['{}_date_shift_four'.format(i)] = auxiliary_viol_df.groupby(['block_lot'])['{}_date'.format(i)].shift(+4)

    viol_df = viol_df.merge(auxiliary_viol_df, on = ['block_lot', 'date'], how = 'left')
    viol_df['{}_date_ffill'.format(i)] = viol_df.groupby(['block_lot'])['{}_date'.format(i)].ffill()
    viol_df['{}_date_ffill_shift'.format(i)] = viol_df.groupby(['block_lot'])['{}_date_shift'.format(i)].ffill()
    viol_df['{}_ffill_shift_two'.format(i)] = viol_df.groupby(['block_lot'])['{}_date_shift_two'.format(i)].ffill()
    viol_df['{}_ffill_shift_three'.format(i)] = viol_df.groupby(['block_lot'])['{}_date_shift_three'.format(i)].ffill()
    viol_df['{}_ffill_shift_four'.format(i)] = viol_df.groupby(['block_lot'])['{}_date_shift_four'.format(i)].ffill()

    viol_df['days_since_{}'.format(i)] = viol_df['date'] - viol_df['{}_date_ffill'.format(i)]
    viol_df['days_since_{}_shift'.format(i)] = viol_df['date'] - viol_df['{}_date_ffill_shift'.format(i)]
    viol_df['days_since_{}_shift_two'.format(i)] = viol_df['date'] - viol_df['{}_ffill_shift_two'.format(i)]
    viol_df['days_since_{}_shift_three'.format(i)] = viol_df['date'] - viol_df['{}_ffill_shift_three'.format(i)]
    viol_df['days_since_{}_shift_four'.format(i)] = viol_df['date'] - viol_df['{}_ffill_shift_four'.format(i)]
    
    viol_df['days_since_{}'.format(i)] =  viol_df['days_since_{}'.format(i)].dt.days
    viol_df['days_since_{}_shift'.format(i)] =  viol_df['days_since_{}_shift'.format(i)].dt.days
    viol_df['days_since_{}_shift_two'.format(i)] =  viol_df['days_since_{}_shift_two'.format(i)].dt.days
    viol_df['days_since_{}_shift_three'.format(i)] =  viol_df['days_since_{}_shift_three'.format(i)].dt.days
    viol_df['days_since_{}_shift_four'.format(i)] = viol_df['days_since_{}_shift_four'.format(i)].dt.days   
    

    viol_df['days_since_{}'.format(i)] =  viol_df['days_since_{}'.format(i)].replace(np.nan, 0)
    viol_df['days_since_{}_shift'.format(i)] =  viol_df['days_since_{}_shift'.format(i)].replace(np.nan, 0)
    viol_df['days_since_{}_shift_two'.format(i)] =  viol_df['days_since_{}_shift_two'.format(i)].replace(np.nan, 0)
    viol_df['days_since_{}_shift_three'.format(i)] =  viol_df['days_since_{}_shift_three'.format(i)].replace(np.nan, 0)
    viol_df['days_since_{}_shift_four'.format(i)] = viol_df['days_since_{}_shift_four'.format(i)].replace(np.nan, 0)

    viol_df = viol_df.drop([
        '{}_date'.format(i),
        '{}_date_shift'.format(i),
        '{}_date_shift_two'.format(i),
        '{}_date_shift_three'.format(i),
        '{}_date_shift_four'.format(i),
        '{}_date_ffill'.format(i),
        '{}_date_ffill_shift'.format(i),
        '{}_ffill_shift_two'.format(i),
        '{}_ffill_shift_three'.format(i),
        '{}_ffill_shift_four'.format(i)],
        axis = 1)


# B) # C) creating number of viol and cummulative violations till date for 5 most recent violations ocurred  
viol_df['dummy'] = 1
pivot_viol_df = pd.pivot_table(viol_df, index=['block_lot','date',], columns=['NOV Category Description'], values=["dummy"], aggfunc=np.sum)
pivot_viol_df = pivot_viol_df.reset_index(drop = False)
pivot_viol_df.columns = pivot_viol_df.columns.droplevel(0)
pivot_viol_df.columns.values[[0, 1]] = ['block_lot', 'date']

for i in viol_df['NOV Category Description'].unique():
    print(i)
    pivot_viol_df['N_{}_violations_current'.format(i)] = pivot_viol_df[i]
    pivot_viol_df['N_{}_violations_previous'.format(i)] = pivot_viol_df.groupby(['block_lot'])[i].shift(+1)
    pivot_viol_df['N_{}_violations_previous_shift_one'.format(i)] = pivot_viol_df.groupby(['block_lot'])[i].shift(+2)
    pivot_viol_df['N_{}_violations_previous_shift_two'.format(i)] = pivot_viol_df.groupby(['block_lot'])[i].shift(+3)
    pivot_viol_df['N_{}_violations_previous_shift_three'.format(i)] = pivot_viol_df.groupby(['block_lot'])[i].shift(+4)
    pivot_viol_df['N_{}_violations_previous_shift_four'.format(i)] = pivot_viol_df.groupby(['block_lot'])[i].shift(+5)

    pivot_viol_df['{}_cumsum_till_date'.format(i)] = pivot_viol_df.groupby(['block_lot'])['N_{}_violations_current'.format(i)].cumsum()
    pivot_viol_df['{}_cumsum_till_date_shift'.format(i)] = pivot_viol_df.groupby(['block_lot'])['N_{}_violations_previous'.format(i)].cumsum()
    pivot_viol_df['{}_cumsum_till_date_shift_two'.format(i)] = pivot_viol_df.groupby(['block_lot'])['N_{}_violations_previous_shift_one'.format(i)].cumsum()
    pivot_viol_df['{}_cumsum_till_date_shift_three'.format(i)] = pivot_viol_df.groupby(['block_lot'])['N_{}_violations_previous_shift_two'.format(i)].cumsum()
    pivot_viol_df['{}_cumsum_till_date_shift_four'.format(i)] = pivot_viol_df.groupby(['block_lot'])['N_{}_violations_previous_shift_three'.format(i)].cumsum()
    
    pivot_viol_df['{}_cumsum_till_date'.format(i)] = pivot_viol_df.groupby(['block_lot'])['{}_cumsum_till_date'.format(i)].ffill()
    pivot_viol_df['{}_cumsum_till_date_shift'.format(i)] = pivot_viol_df.groupby(['block_lot'])['{}_cumsum_till_date_shift'.format(i)].ffill()
    pivot_viol_df['{}_cumsum_till_date_shift_two'.format(i)] = pivot_viol_df.groupby(['block_lot'])['{}_cumsum_till_date_shift_two'.format(i)].ffill()
    pivot_viol_df['{}_cumsum_till_date_shift_three'.format(i)] = pivot_viol_df.groupby(['block_lot'])['{}_cumsum_till_date_shift_three'.format(i)].ffill()
    pivot_viol_df['{}_cumsum_till_date_shift_four'.format(i)] = pivot_viol_df.groupby(['block_lot'])['{}_cumsum_till_date_shift_four'.format(i)].ffill()
    

    viol_df =  pd.DataFrame.merge(viol_df,
                          pivot_viol_df[[
                              'block_lot', 'date',
                              'N_{}_violations_current'.format(i),

                              'N_{}_violations_previous'.format(i),
                              'N_{}_violations_previous_shift_one'.format(i),
                              'N_{}_violations_previous_shift_two'.format(i),
                              'N_{}_violations_previous_shift_three'.format(i),
                              'N_{}_violations_previous_shift_four'.format(i),
                            '{}_cumsum_till_date'.format(i),
                            '{}_cumsum_till_date_shift'.format(i),
                            '{}_cumsum_till_date_shift_two'.format(i),
                            '{}_cumsum_till_date_shift_three'.format(i),
                            '{}_cumsum_till_date_shift_four'.format(i)]],
                          on=['block_lot', 'date'],
                          how='left')



# Modelling pipeline

### defining the target variables


In [ ]:
# creating target variable for MODELING SCENARIO A: modelling the possibility of any violation happening in the next 60 days 

auxiliary_df = viol_df[['block_lot', 'date']].drop_duplicates()
auxiliary_df['date_shift'] = auxiliary_df.groupby(['block_lot'])['date'].shift(-1)
auxiliary_df['days_difference'] = auxiliary_df['date_shift'] - auxiliary_df['date']  
auxiliary_df['days_difference'] = auxiliary_df['days_difference'].dt.days
# replacing nans with zeroes. NaN values appear in block_lot -date scenarios where there is no info about the next vilation at the blocklot
auxiliary_df['days_difference'] = np.where(auxiliary_df['days_difference'].isna(), 0, auxiliary_df['days_difference'])
# creating the target variable for classification
auxiliary_df['violation_in_following_120_days'] = np.where(auxiliary_df['days_difference'] <= 120, 1,0)
# # removing from modelling most recent scenarios (small number of cases)- for these we are not sure if the violation will happen or not in the following period because this perios is still happening
# auxiliary_df = auxiliary_df[auxiliary_df['date'] <= auxiliary_df['date'].max() - timedelta(days=60)]
viol_df = viol_df.merge(auxiliary_df[['violation_in_following_120_days', 'block_lot', 'date']], on = ['block_lot', 'date'], how = 'left')


# creating target variable for MODELING SCENARIO B: modelling the expected ammount of any type of violations during the next violation-ocurring inspection
auxiliary_df=  viol_df.groupby(['block_lot', 'date']).size().reset_index(name='counts')
auxiliary_df['N_violations_next_insp'] = auxiliary_df.groupby(['block_lot'])['counts'].shift(-1)
# replacing nans with zeroes. NaN values appear in block_lot -date scenarios where there is no info about the next vilation at the blocklot
auxiliary_df['N_violations_next_insp'] = np.where(auxiliary_df['N_violations_next_insp'].isna(), 0,\
                                                  auxiliary_df['N_violations_next_insp'])
# removing from modelling most recent scenarios (small number of cases)- for these we are not sure if the violation will happen or not in the following period because this perios is still happening
auxiliary_df = auxiliary_df[auxiliary_df['date'] <= auxiliary_df['date'].max() - timedelta(days=60)]
auxiliary_df[auxiliary_df['block_lot'] == '3547_018B']
viol_df = viol_df.merge(auxiliary_df[['N_violations_next_insp', 'block_lot', 'date']], on = ['block_lot', 'date'], how = 'left')

# Standardizing the dataframe after all of the feature engineering so far has been done
viol_df = viol_df.drop_duplicates(subset = ['block_lot', 'date'])



### Train test validation rules

In [ ]:
# dividing the dataset into scoring, testing, validation and training
# time series adjusted k fold cross validation (https://stats.stackexchange.com/a/366288) was not feasible, since we dont have a traditional
# scoring data will be the most recent case of each block lot i violations dataset, plus all blocklots which werent present at this dataset


# temporarily reverse date order
# add the groupby row number
viol_df = viol_df.sort_values(by='date', ascending = False)
viol_df['ordinality'] = viol_df.groupby(['block_lot']).cumcount()+1
viol_df['ordinality']= viol_df['ordinality'][::-1]
viol_df[viol_df['block_lot']== '4710_003'].head(20)

# returning back the normal ascending sort
viol_df = viol_df.sort_values(by='date')

# data split rules for first round of model building
viol_df['train_test_validation_split'] = np.where(viol_df['ordinality'] == 1, 'score_data', np.nan)
viol_df['train_test_validation_split'] = np.where(viol_df['ordinality'] == 2, 'test_data',viol_df['train_test_validation_split'])
viol_df['train_test_validation_split'] = np.where(viol_df['ordinality'] == 3, 'validation_data', viol_df['train_test_validation_split'])
viol_df['train_test_validation_split'] = np.where(viol_df['ordinality'] > 3, 'train_data', viol_df['train_test_validation_split'] )

# data split rules for second round of model building, when the best hyperparameters are known
viol_df['train_score_split'] = np.where(viol_df['ordinality'] == 1, 'score_data', np.nan)
viol_df['train_score_split'] = np.where(viol_df['ordinality'] != 1, 'train_data',viol_df['train_score_split'])



### train test split

In [ ]:
# dropping columns not required for modelling
cols_to_drop = [
    'Block',
    'Lot',
    'Street Number',
    'Street Name',
    'Status',
    'NOV Category Description',
    'Item',
    'Neighborhoods - Analysis Boundaries',
    'Supervisor District',
    'Location',
    'date_shift_one',
    'date_shift_two',
    'date_shift_three',
    'date_shift_four',
    'date_shift_five',
    'dummy',
#     'block_lot',
    'date',
    'ordinality'
]
viol_df = viol_df.drop(cols_to_drop, axis = 1)

# replacing Nans and inf values with zeros. These came up in feature engineering while dividing with zero, NaN and similar
viol_df = viol_df.replace(np.nan, 0)
viol_df = viol_df.replace([np.inf, -np.inf], np.nan)


# subsetting datasets for first round of modelling (for optimal model validation purposes)
train_data = viol_df[viol_df['train_test_validation_split'] == 'train_data']
validation_data = viol_df[viol_df['train_test_validation_split'] == 'validation_data']

# subsetting dataset for second round of modelling (for testing purposes)
train_data_second_model = viol_df[(viol_df['train_test_validation_split'] == 'train_data')\
                                  | (viol_df['train_test_validation_split'] == 'validation_data')]
test_data = viol_df[viol_df['train_test_validation_split'] == 'test_data']


# subsetting datasets for final model (for final scoring purposes)
train_data_final_model = viol_df[viol_df['train_score_split'] == 'train_data']
score_data_final_model = viol_df[viol_df['train_score_split'] == 'score_data']

# seeing shapes 
print("viol_df", viol_df.shape)
print("train_data first model",train_data.shape)
print("train_data_second_model",train_data_second_model.shape)
print("train_data_final_model",train_data_final_model.shape)
print("validation_data",validation_data.shape)
print("test_data",test_data.shape)

In [ ]:
print("score_data_final_model",score_data_final_model.shape)

### scaling datasets

In [ ]:
# specifying columns for scaling 
numerical_predictor_features = [
    'days_since_last_viol',
    'days_since_last_viol_shift',
    'days_since_last_viol_shift_two',
    'days_since_last_viol_shift_three',
    'days_since_last_viol_shift_four',
    'N_violations_current_insp',
    'N_violations_last_insp',
    'N_violations_last_insp_shift',
    'N_violations_last_insp_shift_two',
    'N_violations_last_insp_shift_three',
    'N_violations_last_insp_shift_four',
    'violations_cumsum_till_date',
    'violations_cumsum_till_date_shift',
    'violations_cumsum_till_date_shift_two',
    'violations_cumsum_till_date_shift_three',
    'violations_cumsum_till_date_shift_four',
    'avg_days_between_previous_violations',
    'avg_N_violations_recent_inspections',
    'StDev_days_between_previous_violations',
    'ratio_avg_days_avg_N_violations',
    'days_since_last_viol_ratio_other',
    'days_since_last_viol_shift_ratio_other',
    'days_since_last_viol_shift_two_ratio_other',
    'days_since_last_viol_shift_three_ratio_other',
    'days_since_last_viol_shift_four_ratio_other',
    'N_violations_current_insp_ratio_other',
    'N_violations_last_insp_ratio_other',
    'N_violations_last_insp_shift_ratio_other',
    'N_violations_last_insp_shift_two_ratio_other',
    'N_violations_last_insp_shift_three_ratio_other',
    'N_violations_last_insp_shift_four_ratio_other',
    'days_since_last_viol_subtracted',
    'days_since_last_viol_shift_subtracted',
    'days_since_last_viol_shift_two_subtracted',
    'days_since_last_viol_shift_three_subtracted',
    'AVG_days_since_columnwise_diff',
    'StDev_days_since_columnwise_diff',
    'N_violations_current_insp_subtracted',
    'N_violations_last_insp_subtracted',
    'N_violations_last_insp_shift_subtracted',
    'N_violations_last_insp_shift_two_subtracted',
    'N_violations_last_insp_shift_three_subtracted',
    'AVG_N_violations_columnwise_diff',
    'StDev_N_violations_columnwise_diff',
    'days_since_last_viol_ratio_previous',
    'days_since_last_viol_shift_ratio_previous',
    'days_since_last_viol_shift_two_ratio_previous',
    'days_since_last_viol_shift_three_ratio_previous',
    'AVG_days_since_columnwise_ratio',
    'StDev_days_since_columnwise_ratio',
    'N_violations_current_insp_ratio_previous',
    'N_violations_last_insp_ratio_previous',
    'N_violations_last_insp_shift_ratio_previous',
    'N_violations_last_insp_shift_two_ratio_previous',
    'N_violations_last_insp_shift_three_ratio_previous',
    'AVG_N_violations_columnwise_ratio',
    'StDev_N_violations_columnwise_ratio',
    'violations_cumsum_till_date_ratio_previous',
    'violations_cumsum_till_date_shift_ratio_previous',
    'violations_cumsum_till_date_shift_two_ratio_previous',
    'violations_cumsum_till_date_shift_three_ratio_previous',
    'AVG_cumsum_columnwise_ratio',
    'StDev_cumsum_columnwise_ratio',
    'ratio_days_since_N_violations_last_insp',
    'ratio_days_since_N_violations_last_insp_shift',
    'ratio_days_since_N_violations_last_insp_shift_two',
    'ratio_days_since_N_violations_last_insp_shift_three',
    'ratio_days_since_N_violations_last_insp_shift_four',
    'AVG_columnwise_ratio_days_since_N_violations',
    'StDev_columnwise_ratio_days_since_N_violations',
    'days_since_building section',
    'days_since_building section_shift',
    'days_since_building section_shift_two',
    'days_since_building section_shift_three',
    'days_since_building section_shift_four',
    'days_since_other section',
    'days_since_other section_shift',
    'days_since_other section_shift_two',
    'days_since_other section_shift_three',
    'days_since_other section_shift_four',
    'days_since_fire section',
    'days_since_fire section_shift',
    'days_since_fire section_shift_two',
    'days_since_fire section_shift_three',
    'days_since_fire section_shift_four',
    'days_since_sanitation section',
    'days_since_sanitation section_shift',
    'days_since_sanitation section_shift_two',
    'days_since_sanitation section_shift_three',
    'days_since_sanitation section_shift_four',
    'days_since_interior surfaces section',
    'days_since_interior surfaces section_shift',
    'days_since_interior surfaces section_shift_two',
    'days_since_interior surfaces section_shift_three',
    'days_since_interior surfaces section_shift_four',
    'days_since_plumbing and electrical section',
    'days_since_plumbing and electrical section_shift',
    'days_since_plumbing and electrical section_shift_two',
    'days_since_plumbing and electrical section_shift_three',
    'days_since_plumbing and electrical section_shift_four',
    'days_since_security requirements section',
    'days_since_security requirements section_shift',
    'days_since_security requirements section_shift_two',
    'days_since_security requirements section_shift_three',
    'days_since_security requirements section_shift_four',
    'days_since_smoke detection section',
    'days_since_smoke detection section_shift',
    'days_since_smoke detection section_shift_two',
    'days_since_smoke detection section_shift_three',
    'days_since_smoke detection section_shift_four',
    'days_since_lead section',
    'days_since_lead section_shift',
    'days_since_lead section_shift_two',
    'days_since_lead section_shift_three',
    'days_since_lead section_shift_four',
    'days_since_hco',
    'days_since_hco_shift',
    'days_since_hco_shift_two',
    'days_since_hco_shift_three',
    'days_since_hco_shift_four',
    'N_building section_violations_current',
    'N_building section_violations_previous',
    'N_building section_violations_previous_shift_one',
    'N_building section_violations_previous_shift_two',
    'N_building section_violations_previous_shift_three',
    'N_building section_violations_previous_shift_four',
    'building section_cumsum_till_date',
    'building section_cumsum_till_date_shift',
    'building section_cumsum_till_date_shift_two',
    'building section_cumsum_till_date_shift_three',
    'building section_cumsum_till_date_shift_four',
    'N_other section_violations_current',
    'N_other section_violations_previous',
    'N_other section_violations_previous_shift_one',
    'N_other section_violations_previous_shift_two',
    'N_other section_violations_previous_shift_three',
    'N_other section_violations_previous_shift_four',
    'other section_cumsum_till_date',
    'other section_cumsum_till_date_shift',
    'other section_cumsum_till_date_shift_two',
    'other section_cumsum_till_date_shift_three',
    'other section_cumsum_till_date_shift_four',
    'N_fire section_violations_current',
    'N_fire section_violations_previous',
    'N_fire section_violations_previous_shift_one',
    'N_fire section_violations_previous_shift_two',
    'N_fire section_violations_previous_shift_three',
    'N_fire section_violations_previous_shift_four',
    'fire section_cumsum_till_date',
    'fire section_cumsum_till_date_shift',
    'fire section_cumsum_till_date_shift_two',
    'fire section_cumsum_till_date_shift_three',
    'fire section_cumsum_till_date_shift_four',
    'N_sanitation section_violations_current',
    'N_sanitation section_violations_previous',
    'N_sanitation section_violations_previous_shift_one',
    'N_sanitation section_violations_previous_shift_two',
    'N_sanitation section_violations_previous_shift_three',
    'N_sanitation section_violations_previous_shift_four',
    'sanitation section_cumsum_till_date',
    'sanitation section_cumsum_till_date_shift',
    'sanitation section_cumsum_till_date_shift_two',
    'sanitation section_cumsum_till_date_shift_three',
    'sanitation section_cumsum_till_date_shift_four',
    'N_interior surfaces section_violations_current',
    'N_interior surfaces section_violations_previous',
    'N_interior surfaces section_violations_previous_shift_one',
    'N_interior surfaces section_violations_previous_shift_two',
    'N_interior surfaces section_violations_previous_shift_three',
    'N_interior surfaces section_violations_previous_shift_four',
    'interior surfaces section_cumsum_till_date',
    'interior surfaces section_cumsum_till_date_shift',
    'interior surfaces section_cumsum_till_date_shift_two',
    'interior surfaces section_cumsum_till_date_shift_three',
    'interior surfaces section_cumsum_till_date_shift_four',
    'N_plumbing and electrical section_violations_current',
    'N_plumbing and electrical section_violations_previous',
    'N_plumbing and electrical section_violations_previous_shift_one',
    'N_plumbing and electrical section_violations_previous_shift_two',
    'N_plumbing and electrical section_violations_previous_shift_three',
    'N_plumbing and electrical section_violations_previous_shift_four',
    'plumbing and electrical section_cumsum_till_date',
    'plumbing and electrical section_cumsum_till_date_shift',
    'plumbing and electrical section_cumsum_till_date_shift_two',
    'plumbing and electrical section_cumsum_till_date_shift_three',
    'plumbing and electrical section_cumsum_till_date_shift_four',
    'N_security requirements section_violations_current',
    'N_security requirements section_violations_previous',
    'N_security requirements section_violations_previous_shift_one',
    'N_security requirements section_violations_previous_shift_two',
    'N_security requirements section_violations_previous_shift_three',
    'N_security requirements section_violations_previous_shift_four',
    'security requirements section_cumsum_till_date',
    'security requirements section_cumsum_till_date_shift',
    'security requirements section_cumsum_till_date_shift_two',
    'security requirements section_cumsum_till_date_shift_three',
    'security requirements section_cumsum_till_date_shift_four',
    'N_smoke detection section_violations_current',
    'N_smoke detection section_violations_previous',
    'N_smoke detection section_violations_previous_shift_one',
    'N_smoke detection section_violations_previous_shift_two',
    'N_smoke detection section_violations_previous_shift_three',
    'N_smoke detection section_violations_previous_shift_four',
    'smoke detection section_cumsum_till_date',
    'smoke detection section_cumsum_till_date_shift',
    'smoke detection section_cumsum_till_date_shift_two',
    'smoke detection section_cumsum_till_date_shift_three',
    'smoke detection section_cumsum_till_date_shift_four',
    'N_lead section_violations_current',
    'N_lead section_violations_previous',
    'N_lead section_violations_previous_shift_one',
    'N_lead section_violations_previous_shift_two',
    'N_lead section_violations_previous_shift_three',
    'N_lead section_violations_previous_shift_four',
    'lead section_cumsum_till_date',
    'lead section_cumsum_till_date_shift',
    'lead section_cumsum_till_date_shift_two',
    'lead section_cumsum_till_date_shift_three',
    'lead section_cumsum_till_date_shift_four',
    'N_hco_violations_current',
    'N_hco_violations_previous',
    'N_hco_violations_previous_shift_one',
    'N_hco_violations_previous_shift_two',
    'N_hco_violations_previous_shift_three',
    'N_hco_violations_previous_shift_four',
    'hco_cumsum_till_date',
    'hco_cumsum_till_date_shift',
    'hco_cumsum_till_date_shift_two',
    'hco_cumsum_till_date_shift_three',
    'hco_cumsum_till_date_shift_four']


# SCALING DATA 
# first model
scaler = StandardScaler()
scaler = scaler.fit(train_data.loc[:, numerical_predictor_features])
train_scaled_features = scaler.transform(train_data.loc[:, numerical_predictor_features])
train_scaled_features = pd.DataFrame(train_scaled_features)
train_scaled_features.columns = numerical_predictor_features
train_data = train_data.drop(numerical_predictor_features, axis=1).reset_index(level=0, drop=True)
train_data = pd.concat((train_data, train_scaled_features), axis=1)

validation_scaled_features = scaler.transform(validation_data.loc[:, numerical_predictor_features])
validation_scaled_features = pd.DataFrame(validation_scaled_features)
validation_scaled_features.columns = numerical_predictor_features
validation_data = validation_data.drop(numerical_predictor_features, axis=1).reset_index(level=0, drop=True)
validation_data = pd.concat((validation_data, validation_scaled_features), axis=1)


# second model
scaler_second_model = StandardScaler()
scaler_second_model = scaler_second_model.fit(train_data_second_model.loc[:, numerical_predictor_features])
train_second_scaled_features = scaler_second_model.transform(train_data_second_model.loc[:, numerical_predictor_features])
train_second_scaled_features = pd.DataFrame(train_second_scaled_features)
train_second_scaled_features.columns = numerical_predictor_features
train_data_second_model = train_data_second_model.drop(numerical_predictor_features, axis=1).reset_index(level=0, drop=True)
train_data_second_model = pd.concat((train_data_second_model, train_second_scaled_features), axis=1)

test_scaled_features = scaler_second_model.transform(test_data.loc[:, numerical_predictor_features])
test_scaled_features = pd.DataFrame(test_scaled_features)
test_scaled_features.columns = numerical_predictor_features
test_data = test_data.drop(numerical_predictor_features, axis=1).reset_index(level=0, drop=True)
test_data = pd.concat((test_data, test_scaled_features), axis=1)


# final model
scaler_final_model = StandardScaler()
scaler_final_model = scaler_final_model.fit(train_data_final_model.loc[:, numerical_predictor_features])
train_final_scaled_features = scaler_final_model.transform(train_data_final_model.loc[:, numerical_predictor_features])
train_final_scaled_features = pd.DataFrame(train_final_scaled_features)
train_final_scaled_features.columns = numerical_predictor_features
train_data_final_model = train_data_final_model.drop(numerical_predictor_features, axis=1).reset_index(level=0, drop=True)
train_data_final_model = pd.concat((train_data_final_model, train_final_scaled_features), axis=1)

score_final_scaled_features = scaler_final_model.transform(score_data_final_model.loc[:, numerical_predictor_features])
score_final_scaled_features = pd.DataFrame(score_final_scaled_features)
score_final_scaled_features.columns = numerical_predictor_features
score_data_final_model = score_data_final_model.drop(numerical_predictor_features, axis=1).reset_index(level=0, drop=True)
score_data_final_model = pd.concat((score_data_final_model, score_final_scaled_features), axis=1)


# Classification model

### First model - choosing the optimal parameters

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import xgboost


# define predictor and target feature. So far only categorical features are in the model
predictor_features = numerical_predictor_features
target_feature = 'violation_in_following_120_days'

# create parameter grid
xgb_param_grid ={
    'nthread': [1],  # use maximum number of threads
    'objective': ['binary:logistic'],
    "grow_policy": ["lossguide"],
    "num_class": [1],
    "tree_method": ["hist"],
    "max_depth": [5,8,9,12,15], 
    "min_child_weight": [2,4,6,8],  
    "colsample_bytree": [0.7,0.75,0.8,0.85],
    "gamma": [0, 1],
    "subsample": [0.7,0.8,0.9,0.95],
    'learning_rate': [0.01, 0.07, 0.1],
    'silent': [1],
    'seed': [1337],
    "n_estimators": [10, 15, 20,40,60]
    }


# choosing the first model
best_score = 0
best_params = None
for param in ParameterGrid(xgb_param_grid):
    # fit the model
    model = xgboost.XGBClassifier(**param)
    model.fit(train_data[predictor_features],train_data[target_feature])
    
    # get auc
    predicted_values = model.predict_proba(validation_data[predictor_features])[:,1]
    auc = roc_auc_score(validation_data['violation_in_following_120_days'], predicted_values)
    

    # save if best
    if auc > best_score:
        best_score = auc
        best_params = model.get_xgb_params()



In [ ]:
print(best_params)


### second model - evaulation on test data

In [ ]:
model = xgboost.XGBClassifier(**best_params)
model.fit(train_data_second_model[predictor_features],train_data_second_model[target_feature])
predicted_values = model.predict_proba(test_data[predictor_features])[:,1]
auc = roc_auc_score(test_data['violation_in_following_120_days'], predicted_values)



In [ ]:
# score data
scored_values = model.predict_proba(score_data_final_model[predictor_features])[:,1] 



In [ ]:
print("test auc", auc)


In [ ]:
# getting accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(test_data['violation_in_following_120_days'], predicted_values)
accuracy_scores = []
for thresh in thresholds:
    accuracy_scores.append(accuracy_score(test_data['violation_in_following_120_days'], 
                                         [1 if m > thresh else 0 for m in predicted_values]))

accuracies = np.array(accuracy_scores)
max_accuracy = accuracies.max() 
max_accuracy_threshold =  thresholds[accuracies.argmax()]

In [ ]:
print(max_accuracy)

## Regression model

### first model

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_absolute_error
import xgboost


# define predictor and target feature. So far only categorical features are in the model
predictor_features = numerical_predictor_features
target_feature = 'N_violations_next_insp'
# create parameter grid
xgb_param_grid ={
    'nthread': [1],  # use maximum number of threads
    'objective': ['reg:squarederror'],
    "grow_policy": ["lossguide"],
    # "num_class": [1],
    "tree_method": ["hist"],
    "max_depth": [4,5,6,7,8], 
    "min_child_weight": [2,4,6,8],  
    "colsample_bytree": [0.7,0.75,0.8,0.85],
    "gamma": [0, 1],
    "subsample": [0.7,0.8,0.9,0.95],
    'learning_rate': [0.01, 0.07, 0.1],
    'silent': [1],
    'seed': [1337],
    "n_estimators": [10, 15, 20,40,60]
    }


# choosing the first model
regressor_best_score = 1000
regressor_best_params = None
for param in ParameterGrid(xgb_param_grid):
    # fit the model
    model = xgboost.XGBRegressor(**param)
    model.fit(train_data[predictor_features],train_data[target_feature])
    
    # get auc
    predicted_values = model.predict(validation_data[predictor_features])
    mae = mean_absolute_error(validation_data[target_feature], predicted_values)
    

    # save if best
    if mae < regressor_best_score:
        regressor_best_score = mae
        regressor_best_params = model.get_xgb_params()







In [ ]:
regressor_model = xgboost.XGBRegressor(**regressor_best_params)
regressor_model.fit(train_data_second_model[predictor_features],train_data_second_model[target_feature])
regressor_predicted_values = regressor_model.predict(test_data[predictor_features])
mae = mean_absolute_error(test_data[target_feature], regressor_predicted_values)

In [ ]:
print("test mae", mae)

In [ ]:
print(test_data[target_feature].mean())